This is the prototype of the Mean Field

There will be three main modules:

- 1) Agent

- 2) MeanField

- 3) BlackBoard

In [162]:
import numpy as np
import scipy as sp
import ode
from sliding_window import *

### Agent

In [163]:
class Agent:
    
    def __init__(self, mean_field, blackboad, state_indices, control_indices):
        '''
        state_indices (list of integers): This list tells which states pertain to this agent. e.g. [1,2] would 
        tell us that states 1 and 2 pertain to this agent.
        
        A word on notation:  The notation used for the methods of the agent is:  
            - If it is a partial derivative: <denominator>_rhs_H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g., 
            "qp_rhs_H_l_u" denotes the partial derivative with respect to q and p of the terms in the local Hamiltonian that contain control variables.
            - If it is a hamiltonian: H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g. "H_mf_nou" denotes the mean field hamiltonian
            with terms not containing u.
        '''
        self.state_indices = state_indices
        self.control_indices = control_indices
        self.mf = mean_field
        self.bb = blackboard
        self.u_l_vec = None
        self.qp_l_vec = None
        self.u_l_dict = None
        self.qp_l_dict = None
    
    def L_l(self, q, q_dot, u):
        return
    
    def L_l_q_dot(self, t, q, q_dot, u, **kwargs):
        return
    
    def H_l_nou(self, q, p, lambda_l):
        return 

    def qp_rhs_H_l_nou(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot = np.zeros(np.shape(p))
        p_D_dot = np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_l_nou(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        returnurn -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_u(self, q, p, u):
        return 
    
    def qp_rhs_H_l_u(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_l_u(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def qp_rhs_H_s_nou(self, t, qp_vec, **kwargs):
        # This will include a term for qp_rhs_l and a term for qp_rhs_MF
        # kwargs should have quenched values
        qp_vec_quenched = self.bb.construct_q_p(qp_vec, self)
        # yes it is really slow, but we read from blackboard every time we evaluate this
        qp_rhs_H_s = (1-alpha)*self.qp_rhs_H_l_nou(qp_vec) + alpha*mf.qp_rhs_H_mf_nou(qp_vec)
        # must return result in qp_vec format because ode rk23 will call this
        # qp_rhs_H_s should look like this: np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
        return qp_rhs_H_s
       
    def qp_rhs_H_s_u(self, t, qp_vec, **kwargs):
        # This will include a term for qp_rhs_l and a term for qp_rhs_MF: 
        qp_rhs_H_s = (1-alpha)*qp_rhs_l(self, qp_vec) + alpha*qp_rhs_MF()
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_s_nou(self, t, u_vec, **kwargs):
        # This will include a term for qp_rhs_l and a term for qp_rhs_MF: 
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def u_rhs_H_s_u(self, t, u_vec, **kwargs):
        # This will include a term for qp_rhs_l and a term for qp_rhs_MF: 
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_D():
        pass
        
    def L_l_D():
        pass
        
    def L_l_D_q_Dot():
        pass

    def qp_rhs(self, t, qp_vec, **kwargs):
        # 
        q=qp_vec[]
        p=qp_vec[]

        qp_rhs_H_mf = self.mf.qp_rhs_H_mf()
        p_rhs_H_mf = qp_rhs_H_mf[:state_dim]
        q_rhs_H_mf = qp_rhs_H_mf[state_dim:]
        
        qp_rhs_H_l = self.qp_rhs_H_l()
        q_rhs_H_l = qp_rhs_H_l[:state_dim]
        p_rhs_H_l = qp_rhs_H_l[state_dim:]
        
        q_l_dot = gamma*q_rhs_H_mf + (1-gamma)*q_rhs_H_l
        p_mf_dot = p_rhs_H_mf ## assuming the user gives us this but waiting for response from Shen about this
        p_l_dot = -1*p_rhs_H_l
        
        return np.concatenate([q_l_dot, p_mf_dot, p_l_dot])
    
    def qp_rhs_H_l()

    def p_rhs_H_l(self, q_l, p_l, u_l):
        q_l_dot = 
        return q_l_dot
    
    def q_rhs_H_l(self, q_l, p_l, u_l):
        return 
    
    def u_rhs():
        pass
    
    # Inputs for numerical propagator
    q_0 = np.array([0])
    p_0 = np.array([0])
    q_D = np.array([0])
    p_D = np.array([0])
    u_0 = np.array([0])
    qpu_vec = np.hstack([q_0, p_0, q_D, p_D, u_0])
    state_dim = 1 
    Gamma = 1 
       
    # Inputs for numerical integration
    integrateTol = 10**-3
    integrateMaxIter = 40
       
    # Inputs for sliding window
    t_0 = 0 
    T =  2
    K=1 
    t_terminal = 2 
    n_s = 10



## MeanField

In [166]:
class MeanField:
    
    def __init__(self):
        pass
    
    def H_MF_nou(self, q, p):
        pass

    def H_MF_u(self, q, p):
        pass
        
    def qp_rhs_H_mf(self, t, qp_vec, **kwargs):
        # remember that we want to propagate as much as possible together in the same rhs function for numerical purposes
        
    def q_rhs_H_mf_u(self, t, qp_vec, **kwargs):
        pass
    
    def p_rhs_H_mf(self, q_l, p_mf, p_l, u_l):
        return self.p_rhs_H_mf_u(q_l, p_mf, u_l) + sum([self.p_rhs_H_mf_nou()*u for i,j in zip()])
        pass

    def p_rhs_H_mf_nou(self, t, qp_vec, **kwargs):
        pass

    def q_rhs_H_mf_nou(self, t, qp_vec, **kwargs):
        pass

    def p_rhs_H_mf_nou(self, t, qp_vec, **kwargs):
        pass
    
    def p_rhs_H_mf_u():
        pass
          

In [176]:
a=[3,5,2]
b=[2]
g=[a,b]
set([x for g_i in g for x in g_i])

{2, 3, 5}

In [165]:
class Synchronizer:
    
    def __init__(self, agents, blackboard):
        self.agents = agents  # list of all agents.  list with elements of class Agent
        self.bb = blackboard  # instance of class blackboard  
    
    def synchronize():
        # run synchronization
        for agent in self.agents:
            '''     
            1) run synchronized propagation - I think we only need one Agent instead of SlidingWindow now

            For each of the above 2 steps:
                - create sliding window instance
                - call "propagate_dynamics" on the sliding window instance

            get quenched values from blackboard
            '''
            # determine dimensions of p_l and p_MF, and then create
            # dimensions of p_l are determined by number of states for this agent 
            # dimensions of p_MF 
            state_dim_l = len(agent.state_indices)
            # set difference between all states in all agents, and states in current agent
            state_dim_mf = set([agent_i.state_indices for agent_i in self.agents for agent_i.state_indices in agent_i]) - set(agent.state_indices)
            # run propagation with keyword arguments state_dim_l, state_dim_mf
            agent.sliding_window = 
            qp_vec = self.bb.construct_q_p(agent) # quench the necessary values to be quenched
            qp_vec = qp_rhs_H_s_nou(qp_vec)

### Blackboard

In [154]:
class Blackboard:
    
    def __init__(self):
        '''
#         state_agent_mapping is dictionary with keys: object of class Agent, 
#               values: list of integers state_indices state_indices
        '''
#         self.state_agent_mapping={}
        '''       
        q_p_u_dict is a dictionary which maps 'q', 'p', 'u', to a dictionary of index-value pairs: local values for q, p, and u for this agent.  
                        blackboard holds all of the most recent local values, e.g.
                        {'q': {'1':3, '2':0}, 'p': {'1':3, '2': 2}, 'u': {'1': 0}}
                        It doesn't care which agent updated them most recently.  It only needs to know which values to update.
        q_p_u_dict initially will be filled 
        '''
        self.q_p_u_dict={'q':{}, 'p':{}, 'u':{}}
        self.agents=[]
    
    def construct_p_q(self, agent):
        '''
        Inputs:
            agent (instance of class Agent): This is the agent for which we are constructing p and q.
        
        Outputs:
            q_p_u_dict (dictionary): This dictionary holds all of the q, p, and u values for all of the agents
        '''
        assert agent in self.agents, 'this agent\'s values have not been added to the blackboard.  please call "self.update_q_p_u_dict(<agent>)" before calling this method'
        q_p_u_dict = {'q':{}, 'p':{}, 'u':{}}
        # find which states pertain to this agent
        # for the states that don't, get the values from q_p_u_dict and put them into qp_vec
        for state_ix in agent.state_indices:
            q_p_u_dict['q'][str(state_ix)] = self.q_p_u_dict['q'][str(state_ix)]
            q_p_u_dict['p'][str(state_ix)] = self.q_p_u_dict['p'][str(state_ix)]

        for control_ix in agent.control_indices:
            q_p_u_dict['u'][str(control_ix)] = self.q_p_u_dict['u'][str(control_ix)]

        # for state indices which are not part of agent.state_indices, fill in from blackboard q_p_u_dict
        assert len(set([str(x) for x in agent.control_indices]) - set(self.q_p_u_dict['u'].keys())) == 0, ' there should not be any controls in agent that are not in blackboard '
        assert len(set([str(x) for x in agent.state_indices]) - set(self.q_p_u_dict['q'].keys())) == 0, ' there should not be any states in agent that are not in blackboard '

        # blackboard - agent
        quenched_state_indices = set([str(x) for x in agent.state_indices]) - set(self.q_p_u_dict['q'].keys())
        # fill in with values from blackboard
        for state_ix in quenched_state_indices:
            q_p_u_dict['q'][str(state_ix)] = self.q_p_u_dict['q'][str(state_ix)]
            q_p_u_dict['p'][str(state_ix)] = self.q_p_u_dict['p'][str(state_ix)]
            
        return q_p_u_dict
    
    def update_q_p_u_dict(self, agent):
        '''  This method should be called after local propagation of each agent
        Inputs:
            agent (instance of class Agent): this is the agent whos values we are updating
        Outputs:
            No outputs.  This method just updates the attributes of the blackboard,
            just update the dictionary, agent_q_p_u_dict.
        '''
        # determine which states pertain to this agent and replace the old values with new
        # assert that the number of states is same as number of costates:
        assert len(agent.q_p_u_dict['q']) == len(agent.q_p_u_dict['p']), 'Number of states is not same as number of costates'
        
        for state_ix in agent.state_indices:
            self.q_p_u_dict['q'][str(state_ix)] = agent.q_p_u_dict['q'][str(state_ix)]
            self.q_p_u_dict['p'][str(state_ix)] = agent.q_p_u_dict['p'][str(state_ix)]
            
        for control_ix in agent.control_indices:
            self.q_p_u_dict['u'][str(control_ix)] = agent.q_p_u_dict['u'][str(control_ix)]
        self.agents.append(agent)
    # Look at what things the Blackboard needs
    # It at least needs a mapping that tells which states pertain to which agent
    # It needs to keep track of all states, costates, controls for all agents so that we can plug in values when necessary

#### small test

In [155]:
myAgent=Agent(state_indices=[1,2], control_indices=[1,2])
myAgent.q_p_u_dict={}
myAgent.q_p_u_dict['q']={}
myAgent.q_p_u_dict['q']['1']=0
myAgent.q_p_u_dict['q']['2']=1

myAgent.q_p_u_dict['p']={}
myAgent.q_p_u_dict['p']['1']=0
myAgent.q_p_u_dict['p']['2']=1

myAgent.q_p_u_dict['u']={}
myAgent.q_p_u_dict['u']['1']=0
myAgent.q_p_u_dict['u']['2']=1

bb=Blackboard()

In [156]:
bb.update_q_p_u_dict(myAgent)

In [157]:
bb.q_p_u_dict

{'p': {'1': 0, '2': 1}, 'q': {'1': 0, '2': 1}, 'u': {'1': 0, '2': 1}}

In [158]:
print myAgent.control_indices
print myAgent.state_indices

[1, 2]
[1, 2]


In [159]:
bb.q_p_u_dict

{'p': {'1': 0, '2': 1}, 'q': {'1': 0, '2': 1}, 'u': {'1': 0, '2': 1}}

In [160]:
q_p_u_dict = bb.construct_p_q(myAgent)

In [161]:
q_p_u_dict

{'p': {'1': 0, '2': 1}, 'q': {'1': 0, '2': 1}, 'u': {'1': 0, '2': 1}}

### What things we need in the blackboard: